In [16]:
import os, subprocess, shutil

# replace this with the nvcc path you posted (we compute CUDA_HOME automatically)
nvcc_path = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc"

if not os.path.exists(nvcc_path):
    raise FileNotFoundError(f"nvcc not found at {nvcc_path} — update nvcc_path if different")

cuda_home = os.path.dirname(os.path.dirname(nvcc_path))
os.environ["CUDA_HOME"] = cuda_home
os.environ["PATH"] = os.path.join(cuda_home, "bin") + os.pathsep + os.environ.get("PATH", "")
# Append existing LD_LIBRARY_PATH if present
os.environ["LD_LIBRARY_PATH"] = os.path.join(cuda_home, "lib64") + os.pathsep + os.environ.get("LD_LIBRARY_PATH", "")

print("CUDA_HOME =", cuda_home)
print("which nvcc ->", shutil.which("nvcc"))

# quick verify (runs in a shell inheriting these env vars)
proc = subprocess.run("/bin/bash -lc 'which nvcc && nvcc --version'", shell=True, capture_output=True, text=True, env=os.environ)
print(proc.stdout)
if proc.returncode != 0:
    print("nvcc failed to run; stderr:\n", proc.stderr)

CUDA_HOME = /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
which nvcc -> /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc
/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0



In [14]:
##NUMPY VERSION IMPLEMENTING ONE TRANSFORMER BLOCK (FORWARD AND BACKPASS)


import numpy as np
np.random.seed(0)

# -------------------- tiny config (single-head for clarity) --------------------
B, T, D = 1, 2, 4        # batch, seq-length, model-dim (small for readability)
d_k = D                  # single head uses full D for simplicity

# -------------------- simple helpers --------------------
def softmax(x, axis=-1):
    e = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e / np.sum(e, axis=axis, keepdims=True)

def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi)*(x + 0.044715*x**3)))

def gelu_grad(x):
    a = np.sqrt(2/np.pi)
    u = a*(x + 0.044715*x**3)
    tanh_u = np.tanh(u)
    sech2 = 1 - tanh_u**2
    du_dx = a*(1 + 3*0.044715*x**2)
    return 0.5*(1 + tanh_u) + 0.5*x*sech2*du_dx

def layer_norm_forward(x, eps=1e-5):
    # x: (B,T,D) per-token LN across features
    mu = x.mean(axis=-1, keepdims=True)
    var = x.var(axis=-1, keepdims=True)
    inv = 1.0/np.sqrt(var + eps)
    x_norm = (x - mu)*inv
    return x_norm, (x, mu, var, inv, eps)

def layer_norm_backward(dy, cache):
    # standard LN backward (per-token)
    x, mu, var, inv, eps = cache
    N = x.shape[-1]
    x_mu = x - mu
    dx_norm = dy
    dvar = np.sum(dx_norm * x_mu * -0.5 * inv**3, axis=-1, keepdims=True)
    dmu = np.sum(dx_norm * -inv, axis=-1, keepdims=True) + dvar * np.sum(-2.0 * x_mu, axis=-1, keepdims=True)/N
    dx = dx_norm * inv + dvar * 2.0 * x_mu / N + dmu / N
    return dx

# -------------------- params (random small init) --------------------
W_Q = np.random.randn(D, D) * 0.1
W_K = np.random.randn(D, D) * 0.1
W_V = np.random.randn(D, D) * 0.1
W_O = np.random.randn(D, D) * 0.1

W1 = np.random.randn(D, 2*D) * 0.1   # FFN expand
b1 = np.zeros(2*D)
W2 = np.random.randn(2*D, D) * 0.1
b2 = np.zeros(D)

gamma1 = np.ones(D); beta1 = np.zeros(D)   # LN after input (scale/shift)
gamma2 = np.ones(D); beta2 = np.zeros(D)   # LN after attn/residual

# -------------------- toy input + target --------------------
x = np.random.randn(B, T, D)
target = np.random.randn(B, T, D)

# -------------------- FORWARD PASS --------------------
cache = {}

# Pre-LN (simplified: LN on input)
x_norm1, ln1_cache = layer_norm_forward(x)                 # x_norm = (x - mu)/sqrt(var)
x_ln = x_ln * gamma1 + beta1                            # scale & shift
cache['ln1'] = (ln1_cache, x_norm1)  # cache both! (CLAUDE CHANGE)


# Q, K, V linear projections
Q = x_ln @ W_Q               # (B,T,D)
K = x_ln @ W_K
V = x_ln @ W_V
cache['Q,K,V'] = (Q, K, V)

# Single-head scaled dot-product attention
scores = Q @ K.transpose(0,2,1) / np.sqrt(d_k)   # (B,T,T)
A = softmax(scores, axis=-1)                      # attention weights
head = A @ V                                      # (B,T,D)
cache['scores,A,head'] = (scores, A, head)

# Output projection
attn_out = head @ W_O            # (B,T,D)
cache['attn_out'] = attn_out

# First residual: x + attn_out
res1 = x + attn_out
cache['res1'] = res1.copy()

# LN before FFN (post-attn LN)
res1_norm, ln2_cache = layer_norm_forward(res1)
res1_ln = res1_norm * gamma2 + beta2
cache['ln2'] = (ln2_cache, res1_norm)

# FFN: expand -> gelu -> project back
h = res1_ln @ W1 + b1       # (B,T,2D)
h_act = gelu(h)
ffn_out = h_act @ W2 + b2   # (B,T,D)
cache['h,h_act,ffn_out'] = (h, h_act, ffn_out)

# final residual -> output
out = res1 + ffn_out         # (B,T,D)

# MSE loss (mean over all elements)
loss = 0.5 * np.mean((out - target)**2)
print("forward loss:", loss)

# -------------------- BACKWARD PASS (manual) --------------------
grads = {name: np.zeros_like(val) for name,val in [
    ("W_Q",W_Q),("W_K",W_K),("W_V",W_V),("W_O",W_O),
    ("W1",W1),("b1",b1),("W2",W2),("b2",b2),
    ("gamma1",gamma1),("beta1",beta1),("gamma2",gamma2),("beta2",beta2)
]}

# dLoss/dout: derivative of 0.5*mean((out-target)^2) -> (out-target)/N_total
N_total = out.size
dout = (out - target) / N_total

# -- final residual out = res1 + ffn_out
dres1 = dout.copy()        # gradient through identity/residual
dffn = dout.copy()         # gradient into FFN output

# -- FFN backward: ffn_out = h_act @ W2 + b2
grads['W2'] += h_act.reshape(-1, h_act.shape[-1]).T @ dffn.reshape(-1, D)   # dW2 = h_act^T * dffn
grads['b2'] += np.sum(dffn, axis=(0,1))                                    # db2 = sum over batch & time

# backprop into h_act
dh_act = dffn @ W2.T                   # (B,T,2D)
dh = dh_act * gelu_grad(h)             # elementwise through GELU

# W1, b1 grads: h = res1_ln @ W1 + b1
grads['W1'] += res1_ln.reshape(-1, D).T @ dh.reshape(-1, 2*D)   # dW1 = res1_ln^T * dh
grads['b1'] += np.sum(dh, axis=(0,1))                         # db1 = sum

# gradient into res1_ln from FFN
dres1_ln_from_ffn = dh @ W1.T      # (B,T,D)

# -- LN2 backward: res1_ln = LN(res1) * gamma2 + beta2
# dgamma2 = sum( dres1_ln_from_ffn * LN(res1) ), dbeta2 = sum(dres1_ln_from_ffn)
# But LN(res1) (normalized values) = (res1 - mu)/sqrt(var) ; we cached ln2_cache for exact backward.
grads['gamma2'] += np.sum(dres1_ln_from_ffn * ((res1 - ln2_cache[1]) * ln2_cache[3]), axis=(0,1))  # dgamma
grads['beta2']  += np.sum(dres1_ln_from_ffn, axis=(0,1))                                           # dbeta

# scale then LN backward
dln2 = dres1_ln_from_ffn * gamma2            # scale by gamma2 (elementwise)
dx_from_ln2 = layer_norm_backward(dln2, ln2_cache)

# total gradient into res1 = from final residual identity + from LN/FFN path
dres1_total = dres1 + dx_from_ln2

# -- residual1: res1 = x + attn_out
# res1 = x + attn_out, so gradient splits:
#   dL/dx = dL/dres1 (identity branch)
#   dL/dattn_out = dL/dres1 (same gradient flows to both inputs)
dx_resid = dres1_total.copy()    # flows to identity branch (input x)
dattn_out = dres1_total.copy()   # flows into attention output

# -- Attention backward
# attn_out = head @ W_O
grads['W_O'] += head.reshape(-1, D).T @ dattn_out.reshape(-1, D)
dhead = dattn_out @ W_O.T        # gradient into head (B,T,D)

# head = A @ V  -> dA and dV
# By chain rule:
#   dL/dA = dL/dhead @ V^T  (where @ is batch matmul)
#   dL/dV = A^T @ dL/dhead
dA = dhead @ V.transpose(0,2,1)          # (B,T,T) = dhead @ V^T  (for single-head shapes)
dV = A.transpose(0,2,1) @ dhead          # (B,T,D) = A^T @ dhead

# softmax backward: A = softmax(scores)
# dScores = A * (dA - sum(dA * A, axis=-1, keepdims=True))
# Softmax Jacobian: dL/dscores[i,j] = A[i,j] * (dL/dA[i,j] - Σ_k dL/dA[i,k] * A[i,k])
# This comes from: dA/dscores = diag(A) - A*A^T per row
tmp = np.sum(dA * A, axis=-1, keepdims=True)
dScores = A * (dA - tmp)

# scores = Q @ K^T / sqrt(d_k)
dQ = dScores @ K / np.sqrt(d_k)          # dQ = dScores @ K / sqrt
dK = dScores.transpose(0,2,1) @ Q / np.sqrt(d_k)  # dK = dScores^T @ Q / sqrt

# Q = x_ln @ W_Q  etc -> compute W grads and dx_ln contribution
grads['W_Q'] += x_ln.reshape(-1, D).T @ dQ.reshape(-1, D)
grads['W_K'] += x_ln.reshape(-1, D).T @ dK.reshape(-1, D)
grads['W_V'] += x_ln.reshape(-1, D).T @ dV.reshape(-1, D)

dx_ln_from_Q = dQ @ W_Q.T
dx_ln_from_K = dK @ W_K.T
dx_ln_from_V = dV @ W_V.T

dx_ln_attn = dx_ln_from_Q + dx_ln_from_K + dx_ln_from_V   # combined gradient into x_ln from attention

# -- LN1 backward: x_ln = LN(x) * gamma1 + beta1
ln1_cache, x_norm1 = cache['ln1']
grads['gamma1'] += np.sum(dx_ln_attn * x_norm1, axis=(0,1))  # cleaner!
grads['beta1']  += np.sum(dx_ln_attn, axis=(0,1))

dln1 = dx_ln_attn * gamma1
dx_from_ln1 = layer_norm_backward(dln1, ln1_cache)

# total gradient to input x = from LN1 path + from residual identity branch
dx_total = dx_from_ln1 + dx_resid

# -------------------- quick prints --------------------
print("\nParameter gradient norms:")
for k in ['W_Q','W_K','W_V','W_O','W1','W2','b1','b2','gamma1','beta1','gamma2','beta2']:
    print(f" {k:6s}: {np.linalg.norm(grads[k]):.6f}")

# -------------------- GRADIENT CHECK --------------------
def check_grad(param_name, param, grad, eps=1e-5):
    """Finite difference check"""
    orig = param.copy()
    errors = []
    for idx in np.ndindex(param.shape):
        param[idx] = orig[idx] + eps
        loss_plus = compute_loss()  # re-run forward
        param[idx] = orig[idx] - eps
        loss_minus = compute_loss()
        param[idx] = orig[idx]
        
        numerical = (loss_plus - loss_minus) / (2 * eps)
        analytical = grad[idx]
        errors.append(abs(numerical - analytical))
    
    print(f"{param_name}: max error = {max(errors):.2e}")


forward loss: 0.3723908986270815

Parameter gradient norms:
 W_Q   : 0.000322
 W_K   : 0.000528
 W_V   : 0.054086
 W_O   : 0.107515
 W1    : 0.087050
 W2    : 0.096397
 b1    : 0.045283
 b2    : 0.347803
 gamma1: 0.011698
 beta1 : 0.009856
 gamma2: 0.007613
 beta2 : 0.008136


In [18]:
%%bash
export CUDA_HOME=/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH
cat > transformer_block.cu <<'EOF'

#include <cuda_runtime.h>
#include <iostream>
#include <cmath>
#include <cstdlib>
#include <ctime>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error at " << __FILE__ << ":" << __LINE__ \
                      << " - " << cudaGetErrorString(err) << std::endl; \
            exit(1); \
        } \
    } while(0)

// ============================================================================
// CUDA KERNELS
// ============================================================================

__global__ void gelu_forward(const float* x, float* y, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        float val = x[idx];
        float a = sqrtf(2.0f / 3.14159265359f);
        float u = a * (val + 0.044715f * val * val * val);
        float tanh_u = tanhf(u);
        y[idx] = 0.5f * val * (1.0f + tanh_u);
    }
}

__global__ void gelu_backward(const float* x, const float* dy, float* dx, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        float val = x[idx];
        float a = sqrtf(2.0f / 3.14159265359f);
        float u = a * (val + 0.044715f * val * val * val);
        float tanh_u = tanhf(u);
        float sech2 = 1.0f - tanh_u * tanh_u;
        float du_dx = a * (1.0f + 3.0f * 0.044715f * val * val);
        float grad = 0.5f * (1.0f + tanh_u) + 0.5f * val * sech2 * du_dx;
        dx[idx] = dy[idx] * grad;
    }
}

__global__ void layernorm_forward(const float* x, float* y, float* mean, float* inv_std,
                                   int BT, int D, float eps) {
    int token_idx = blockIdx.x;
    if (token_idx >= BT) return;
    
    extern __shared__ float shared[];
    
    // Compute mean
    float sum = 0.0f;
    for (int i = threadIdx.x; i < D; i += blockDim.x) {
        sum += x[token_idx * D + i];
    }
    shared[threadIdx.x] = sum;
    __syncthreads();
    
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            shared[threadIdx.x] += shared[threadIdx.x + stride];
        }
        __syncthreads();
    }
    
    float mu = shared[0] / D;
    if (threadIdx.x == 0) mean[token_idx] = mu;
    __syncthreads();
    
    // Compute variance
    sum = 0.0f;
    for (int i = threadIdx.x; i < D; i += blockDim.x) {
        float diff = x[token_idx * D + i] - mu;
        sum += diff * diff;
    }
    shared[threadIdx.x] = sum;
    __syncthreads();
    
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            shared[threadIdx.x] += shared[threadIdx.x + stride];
        }
        __syncthreads();
    }
    
    float var = shared[0] / D;
    float inv = 1.0f / sqrtf(var + eps);
    if (threadIdx.x == 0) inv_std[token_idx] = inv;
    __syncthreads();
    
    // Normalize
    for (int i = threadIdx.x; i < D; i += blockDim.x) {
        y[token_idx * D + i] = (x[token_idx * D + i] - mu) * inv;
    }
}

__global__ void layernorm_backward(const float* dy, const float* x, const float* mean,
                                   const float* inv_std, float* dx, int BT, int D) {
    int token_idx = blockIdx.x;
    if (token_idx >= BT) return;
    
    extern __shared__ float shared[];
    float* s1 = shared;
    float* s2 = &shared[blockDim.x];
    
    float mu = mean[token_idx];
    float inv = inv_std[token_idx];
    int offset = token_idx * D;
    
    float sum1 = 0.0f, sum2 = 0.0f;
    for (int i = threadIdx.x; i < D; i += blockDim.x) {
        float xmu = x[offset + i] - mu;
        sum1 += dy[offset + i];
        sum2 += dy[offset + i] * xmu;
    }
    s1[threadIdx.x] = sum1;
    s2[threadIdx.x] = sum2;
    __syncthreads();
    
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            s1[threadIdx.x] += s1[threadIdx.x + stride];
            s2[threadIdx.x] += s2[threadIdx.x + stride];
        }
        __syncthreads();
    }
    
    float dmu = -inv * s1[0];
    float dvar = -0.5f * inv * inv * inv * s2[0];
    
    for (int i = threadIdx.x; i < D; i += blockDim.x) {
        float xmu = x[offset + i] - mu;
        dx[offset + i] = dy[offset + i] * inv + dvar * 2.0f * xmu / D + dmu / D;
    }
}

__global__ void scale_shift(const float* x, const float* gamma, const float* beta,
                            float* y, int BT, int D) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < BT * D) {
        int d = idx % D;
        y[idx] = x[idx] * gamma[d] + beta[d];
    }
}

__global__ void softmax_forward(const float* x, float* y, int B, int T) {
    int batch = blockIdx.y;
    int row = blockIdx.x;
    if (batch >= B || row >= T) return;
    
    extern __shared__ float shared[];
    int offset = (batch * T + row) * T;
    
    // Max
    float max_val = -1e30f;
    for (int i = threadIdx.x; i < T; i += blockDim.x) {
        max_val = fmaxf(max_val, x[offset + i]);
    }
    shared[threadIdx.x] = max_val;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            shared[threadIdx.x] = fmaxf(shared[threadIdx.x], shared[threadIdx.x + s]);
        }
        __syncthreads();
    }
    max_val = shared[0];
    
    // Exp and sum
    float sum = 0.0f;
    for (int i = threadIdx.x; i < T; i += blockDim.x) {
        float e = expf(x[offset + i] - max_val);
        y[offset + i] = e;
        sum += e;
    }
    shared[threadIdx.x] = sum;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            shared[threadIdx.x] += shared[threadIdx.x + s];
        }
        __syncthreads();
    }
    sum = shared[0];
    
    // Normalize
    for (int i = threadIdx.x; i < T; i += blockDim.x) {
        y[offset + i] /= sum;
    }
}

__global__ void softmax_backward(const float* A, const float* dA, float* dScores,
                                 int B, int T) {
    int batch = blockIdx.y;
    int row = blockIdx.x;
    if (batch >= B || row >= T) return;
    
    extern __shared__ float shared[];
    int offset = (batch * T + row) * T;
    
    float sum = 0.0f;
    for (int i = threadIdx.x; i < T; i += blockDim.x) {
        sum += dA[offset + i] * A[offset + i];
    }
    shared[threadIdx.x] = sum;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            shared[threadIdx.x] += shared[threadIdx.x + s];
        }
        __syncthreads();
    }
    float dot = shared[0];
    
    for (int i = threadIdx.x; i < T; i += blockDim.x) {
        dScores[offset + i] = A[offset + i] * (dA[offset + i] - dot);
    }
}

// Simple matrix multiply kernel: C = A @ B (no transpose)
__global__ void matmul(const float* A, const float* B, float* C, 
                       int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += A[row * K + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

// Matrix multiply with transpose: C = A @ B^T
__global__ void matmul_transB(const float* A, const float* B, float* C,
                              int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += A[row * K + k] * B[col * K + k];
        }
        C[row * N + col] = sum;
    }
}

// Matrix multiply with accumulation: C += A @ B
__global__ void matmul_accum(const float* A, const float* B, float* C,
                             int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; k++) {
            sum += A[row * K + k] * B[k * N + col];
        }
        atomicAdd(&C[row * N + col], sum);
    }
}

__global__ void add_vectors(const float* a, const float* b, float* c, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        c[idx] = a[idx] + b[idx];
    }
}

__global__ void scale_vector(const float* x, float scale, float* y, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        y[idx] = x[idx] * scale;
    }
}

__global__ void elemwise_mul(const float* a, const float* b, float* c, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        c[idx] = a[idx] * b[idx];
    }
}

__global__ void mse_loss(const float* pred, const float* target, float* loss, int N) {
    extern __shared__ float sdata[];
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    float diff = 0.0f;
    if (idx < N) {
        diff = pred[idx] - target[idx];
        diff = diff * diff;
    }
    sdata[threadIdx.x] = diff;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            sdata[threadIdx.x] += sdata[threadIdx.x + s];
        }
        __syncthreads();
    }
    
    if (threadIdx.x == 0) {
        atomicAdd(loss, sdata[0] * 0.5f / N);
    }
}

__global__ void mse_grad(const float* pred, const float* target, float* grad, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        grad[idx] = (pred[idx] - target[idx]) / N;
    }
}

// ============================================================================
// MAIN
// ============================================================================

int main() {
    srand(0);
    
    const int B = 1, T = 2, D = 4;
    const int BT = B * T;
    const float eps = 1e-5f;
    const float sqrt_dk = sqrtf((float)D);
    
    std::cout << "Config: B=" << B << ", T=" << T << ", D=" << D << std::endl;
    
    // Allocate host memory and initialize
    float *h_W_Q = new float[D*D];
    float *h_W_K = new float[D*D];
    float *h_W_V = new float[D*D];
    float *h_W_O = new float[D*D];
    float *h_W1 = new float[D*2*D];
    float *h_W2 = new float[2*D*D];
    float *h_b1 = new float[2*D];
    float *h_b2 = new float[D];
    float *h_gamma1 = new float[D];
    float *h_beta1 = new float[D];
    float *h_gamma2 = new float[D];
    float *h_beta2 = new float[D];
    float *h_x = new float[BT*D];
    float *h_target = new float[BT*D];
    
    // Random init (scale by 0.1)
    for (int i = 0; i < D*D; i++) {
        h_W_Q[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
        h_W_K[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
        h_W_V[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
        h_W_O[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
    }
    for (int i = 0; i < D*2*D; i++) {
        h_W1[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
    }
    for (int i = 0; i < 2*D*D; i++) {
        h_W2[i] = ((float)rand() / RAND_MAX - 0.5f) * 0.2f;
    }
    for (int i = 0; i < 2*D; i++) h_b1[i] = 0.0f;
    for (int i = 0; i < D; i++) h_b2[i] = 0.0f;
    for (int i = 0; i < D; i++) {
        h_gamma1[i] = 1.0f;
        h_beta1[i] = 0.0f;
        h_gamma2[i] = 1.0f;
        h_beta2[i] = 0.0f;
    }
    for (int i = 0; i < BT*D; i++) {
        h_x[i] = ((float)rand() / RAND_MAX - 0.5f) * 2.0f;
        h_target[i] = ((float)rand() / RAND_MAX - 0.5f) * 2.0f;
    }
    
    // Allocate device memory
    float *d_W_Q, *d_W_K, *d_W_V, *d_W_O;
    float *d_W1, *d_W2, *d_b1, *d_b2;
    float *d_gamma1, *d_beta1, *d_gamma2, *d_beta2;
    float *d_x, *d_target;
    
    CUDA_CHECK(cudaMalloc(&d_W_Q, D*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_W_K, D*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_W_V, D*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_W_O, D*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_W1, D*2*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_W2, 2*D*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b1, 2*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b2, D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_gamma1, D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_beta1, D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_gamma2, D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_beta2, D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_x, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_target, BT*D*sizeof(float)));
    
    // Copy to device
    CUDA_CHECK(cudaMemcpy(d_W_Q, h_W_Q, D*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_W_K, h_W_K, D*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_W_V, h_W_V, D*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_W_O, h_W_O, D*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_W1, h_W1, D*2*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_W2, h_W2, 2*D*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b1, h_b1, 2*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b2, h_b2, D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_gamma1, h_gamma1, D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_beta1, h_beta1, D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_gamma2, h_gamma2, D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_beta2, h_beta2, D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_x, h_x, BT*D*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_target, h_target, BT*D*sizeof(float), cudaMemcpyHostToDevice));
    
    // Allocate intermediate activations
    float *d_x_norm1, *d_x_ln, *d_Q, *d_K, *d_V;
    float *d_scores, *d_A, *d_head, *d_attn_out, *d_res1;
    float *d_res1_norm, *d_res1_ln, *d_h, *d_h_act, *d_ffn_out, *d_out;
    float *d_mean1, *d_inv_std1, *d_mean2, *d_inv_std2;
    
    CUDA_CHECK(cudaMalloc(&d_x_norm1, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_x_ln, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_Q, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_K, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_V, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_scores, B*T*T*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_A, B*T*T*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_head, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_attn_out, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_res1, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_res1_norm, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_res1_ln, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_h, BT*2*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_h_act, BT*2*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_ffn_out, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_out, BT*D*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_mean1, BT*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_inv_std1, BT*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_mean2, BT*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_inv_std2, BT*sizeof(float)));
    
    // ========== FORWARD PASS ==========
    std::cout << "\n========== FORWARD PASS ==========\n";
    
    int threads = 256;
    int blocks;
    
    // LayerNorm 1
    layernorm_forward<<<BT, threads, threads*sizeof(float)>>>(
        d_x, d_x_norm1, d_mean1, d_inv_std1, BT, D, eps
    );
    
    blocks = (BT*D + threads - 1) / threads;
    scale_shift<<<blocks, threads>>>(d_x_norm1, d_gamma1, d_beta1, d_x_ln, BT, D);
    
    // Q, K, V projections
    dim3 grid_mm(1, BT);  // For small matrices
    dim3 block_mm(D, 1);
    
    matmul<<<grid_mm, block_mm>>>(d_x_ln, d_W_Q, d_Q, BT, D, D);
    matmul<<<grid_mm, block_mm>>>(d_x_ln, d_W_K, d_K, BT, D, D);
    matmul<<<grid_mm, block_mm>>>(d_x_ln, d_W_V, d_V, BT, D, D);
    
    // Attention scores = Q @ K^T
    dim3 grid_scores(1, BT);
    dim3 block_scores(T, 1);
    matmul_transB<<<grid_scores, block_scores>>>(d_Q, d_K, d_scores, BT, T, D);
    
    // Scale by 1/sqrt(d_k)
    blocks = (B*T*T + threads - 1) / threads;
    scale_vector<<<blocks, threads>>>(d_scores, 1.0f/sqrt_dk, d_scores, B*T*T);
    
    // Softmax
    dim3 grid_soft(T, B);
    softmax_forward<<<grid_soft, threads, threads*sizeof(float)>>>(d_scores, d_A, B, T);
    
    // head = A @ V (simplified for B=1)
    matmul<<<grid_mm, block_mm>>>(d_A, d_V, d_head, BT, D, T);
    
    // Output projection
    matmul<<<grid_mm, block_mm>>>(d_head, d_W_O, d_attn_out, BT, D, D);
    
    // Residual 1
    blocks = (BT*D + threads - 1) / threads;
    add_vectors<<<blocks, threads>>>(d_x, d_attn_out, d_res1, BT*D);
    
    // LayerNorm 2
    layernorm_forward<<<BT, threads, threads*sizeof(float)>>>(
        d_res1, d_res1_norm, d_mean2, d_inv_std2, BT, D, eps
    );
    scale_shift<<<blocks, threads>>>(d_res1_norm, d_gamma2, d_beta2, d_res1_ln, BT, D);
    
    // FFN
    dim3 grid_ffn1(1, BT);
    dim3 block_ffn1(2*D, 1);
    matmul<<<grid_ffn1, block_ffn1>>>(d_res1_ln, d_W1, d_h, BT, 2*D, D);
    
    // Add bias (simplified)
    blocks = (BT*2*D + threads - 1) / threads;
    for (int bt = 0; bt < BT; bt++) {
        add_vectors<<<1, 2*D>>>(d_h + bt*2*D, d_b1, d_h + bt*2*D, 2*D);
    }
    
    // GELU
    blocks = (BT*2*D + threads - 1) / threads;
    gelu_forward<<<blocks, threads>>>(d_h, d_h_act, BT*2*D);
    
    // FFN output
    dim3 grid_ffn2(1, BT);
    dim3 block_ffn2(D, 1);
    matmul<<<grid_ffn2, block_ffn2>>>(d_h_act, d_W2, d_ffn_out, BT, D, 2*D);
    
    for (int bt = 0; bt < BT; bt++) {
        add_vectors<<<1, D>>>(d_ffn_out + bt*D, d_b2, d_ffn_out + bt*D, D);
    }
    
    // Final residual
    blocks = (BT*D + threads - 1) / threads;
    add_vectors<<<blocks, threads>>>(d_res1, d_ffn_out, d_out, BT*D);
    
    // Loss
    float *d_loss;
    CUDA_CHECK(cudaMalloc(&d_loss, sizeof(float)));
    CUDA_CHECK(cudaMemset(d_loss, 0, sizeof(float)));
    
    blocks = (BT*D + threads - 1) / threads;
    mse_loss<<<blocks, threads, threads*sizeof(float)>>>(d_out, d_target, d_loss, BT*D);
    
    float h_loss;
    CUDA_CHECK(cudaMemcpy(&h_loss, d_loss, sizeof(float), cudaMemcpyDeviceToHost));
    
    std::cout << "Forward loss: " << h_loss << std::endl;
    
    // Cleanup
    delete[] h_W_Q; delete[] h_W_K; delete[] h_W_V; delete[] h_W_O;
    delete[] h_W1; delete[] h_W2; delete[] h_b1; delete[] h_b2;
    delete[] h_gamma1; delete[] h_beta1; delete[] h_gamma2; delete[] h_beta2;
    delete[] h_x; delete[] h_target;
    
    cudaFree(d_W_Q); cudaFree(d_W_K); cudaFree(d_W_V); cudaFree(d_W_O);
    cudaFree(d_W1); cudaFree(d_W2); cudaFree(d_b1); cudaFree(d_b2);
    cudaFree(d_gamma1); cudaFree(d_beta1); cudaFree(d_gamma2); cudaFree(d_beta2);
    cudaFree(d_x); cudaFree(d_target); cudaFree(d_loss);
    cudaFree(d_x_norm1); cudaFree(d_x_ln); cudaFree(d_Q); cudaFree(d_K); cudaFree(d_V);
    cudaFree(d_scores); cudaFree(d_A); cudaFree(d_head); cudaFree(d_attn_out);
    cudaFree(d_res1); cudaFree(d_res1_norm); cudaFree(d_res1_ln);
    cudaFree(d_h); cudaFree(d_h_act); cudaFree(d_ffn_out); cudaFree(d_out);
    cudaFree(d_mean1); cudaFree(d_inv_std1); cudaFree(d_mean2); cudaFree(d_inv_std2);
    
    std::cout << "\nDone!\n";
    return 0;
}





EOF

nvcc -std=c++17 transformer_block.cu -o transformer_block_output && ./transformer_block_output

Config: B=1, T=2, D=4

========== FORWARD PASS ==========
Forward loss: 0.597601

Done!
